### Sampling Rate 맞추기

In [ ]:

import soundfile as sf
from scipy.signal import resample
from datasets import Dataset, Audio
import numpy as np
import pandas as pd 
train_df = pd.read_csv('/workspace/kru_data/train.csv')
def wav_info_description(wav_path):
    """
    wav_path 파일을 불러와서 sampling_rate, duration, channels를 출력해줍니다.
    """
    data, samplerate = sf.read(wav_path)
    duration = len(data) / samplerate
    if len(data.shape) == 1:
        channels = 1
    else:
        channels = data.shape[1]

    print(f"파일: {wav_path}")
    print(f"- 샘플링 레이트: {samplerate} Hz")
    print(f"- 길이(초): {duration:.2f}")
    print(f"- 채널 수: {channels}")

wav_info_description(train_df['abs_path'][0])

def wav_load(wav_path, sampling_rate=16000):
    """
    wav_path 파일을 불러와서 sampling_rate로 리샘플링 해줍니다.
    """
    data, samplerate = sf.read(wav_path)
    
    if samplerate != sampling_rate:
        # resample의 두번째 인자는 출력 샘플 수여야 한다.
        # 현재 data의 길이를 기준으로, 타겟 샘플링 레이트에 맞게 리샘플된 길이 계산
        n_samples = int(len(data) * float(sampling_rate) / samplerate)
        data = resample(data, n_samples)
    
    return data

# 샘플 확인
data = wav_load(train_df['abs_path'][0])
print(f"\n로드된 오디오 데이터:")
print(f"  Shape: {data.shape}")
print(f"  Dtype: {data.dtype}")
print(f"  Min: {data.min():.6f}, Max: {data.max():.6f}")

### Whisper 한국어 FE-Tokenizer 확인

In [21]:

from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

In [23]:
input_str = "안녕하세요"
labels = tokenizer.encode(input_str)

decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 안녕하세요
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|><|notimestamps|>안녕하세요<|endoftext|>
Decoded w/out special: 안녕하세요
Are equal:             True


## 허깅페이스 데이터셋 제작

In [ ]:
from utils.data_hg_loader import _prepare_dataframe
import dask.dataframe as dd

train_ddf = dd.read_csv('/workspace/kru_data/train.csv', blocksize='32GB')
test_ddf = dd.read_csv('/workspace/kru_data/test.csv', blocksize='32GB')

train_ddf = _prepare_dataframe(train_ddf)
test_ddf = _prepare_dataframe(test_ddf)


/opt/conda/lib/python3.11/site-packages/dask/dataframe/dask_expr/_collection.py:4399: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('abs_path', 'object'))

  warnings.warn(meta_warning(meta))
/opt/conda/lib/python3.11/site-packages/dask/dataframe/dask_expr/_collection.py:4399: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('abs_path', 'objec